In [1]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

0

In [2]:
final_ka04_df = pd.read_csv('final_sku_data.csv') #44219
print(final_ka04_df.shape)

(44219, 6)


# Data Preparation

In [6]:
# Creating new columns
final_ka04_df['year'] = final_ka04_df['YYYYMM'].astype(str).str[0:4]
final_ka04_df['year'] = final_ka04_df['year'].astype(int)
print("Unique Year ", final_ka04_df['year'].unique())

final_ka04_df['month'] = final_ka04_df['YYYYMM'].astype(str).str[4:]
final_ka04_df['month'] = final_ka04_df['month'].astype(int)
print("Unique month ", final_ka04_df['month'].unique())

final_ka04_df['uid'] = final_ka04_df['DWH_STATE_Cleaned'] + "-" + \
final_ka04_df['ACC'] + "-" + final_ka04_df['Company'] + "-" +  final_ka04_df['SKU_CODE']

print('No of unique uids - ', final_ka04_df['uid'].nunique())
final_ka04_df.head()

Unique Year  [2020]
Unique month  [ 1  2  3  4  5  6  7  8  9 10 11 12]
No of unique uids -  4324


,DWH_STATE_Cleaned,ACC,Company,SKU_CODE,YYYYMM,actual_vol,year,month,uid
0,duty free,711,BAT,B&H FF,202001,3.90,2020,1,duty free-711-BAT-B&H FF
1,duty free,711,BAT,B&H FF,202002,2.26,2020,2,duty free-711-BAT-B&H FF
2,duty free,711,BAT,B&H FF,202003,1.98,2020,3,duty free-711-BAT-B&H FF
3,duty free,711,BAT,B&H FF,202004,0.78,2020,4,duty free-711-BAT-B&H FF
4,duty free,711,BAT,B&H FF,202005,0.68,2020,5,duty free-711-BAT-B&H FF


In [5]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score

def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def exp_smoothing_trend(ts,extra_periods=1,alpha=0.4,beta=0.4,phi=0.9):
 
    """
    This function calculates a forecast with an exponential smoothing + damped trend method.
     
    Inputs
      - ts: the historical values (a list, a numpy array or a pandas series)
      - extra_periods: the number of data points that you want to forecast
      - alpha: the level parameter
      - beta: the trend parameter
      - phi: the trend damping parameter
    """
     
    # Avoid any edition of original list, array or dataframe
    ts = ts.copy()
    
    # Transform ts into list
    try:
        ts = ts.tolist()
    except:
        pass  
       
    # Initialization
    f = [np.nan] # First forecast is set to null value
    a = [ts[0]] # First level defined as the first demand point
    b = [ts[1]-ts[0]] # First trend is computed as the difference between the two first demand points
 
    # Create all the m+1 forecast
    for t in range(1,len(ts)):
        # Update forecast based on last level (a) and trend (b)
        f.append(a[t-1]+phi*b[t-1])
        
        # Update the level based on the new data point
        a.append(alpha*ts[t]+(1-alpha)*(a[t-1]+phi*b[t-1]))
         
        # Update the trend based on the new data point
        b.append(beta*(a[t]-a[t-1])+(1-beta)*phi*b[t-1])
         
    # Forecast for all extra months
    for t in range(extra_periods):
        # Update the forecast as the most up-to-date level + trend
        f.append(a[-1]+phi*b[-1])
        # the level equals the forecast
        a.append(f[-1])
        # Update the trend as the previous trend
        b.append(phi*b[-1])
        # fill in ts by np.nan for easy plotting
        ts.append(np.nan)
         
     
    # Populate table with all the results
    dic = {"demand":ts,"forecast":f,"level":a,"trend":b}
    results = pd.DataFrame.from_dict(dic)
    results.index.name = 'Period'
    results["error"] = results["forecast"] - results["demand"] 
        
    # Return the full data set and an indicator of past MAEP    
    return results  


In [19]:
df_ls = final_ka04_df.copy()
print('Unique UIDs - ', df_ls['uid'].nunique())
df_ls.head()

Unique UIDs -  4324


,DWH_STATE_Cleaned,ACC,Company,SKU_CODE,YYYYMM,actual_vol,year,month,uid
0,duty free,711,BAT,B&H FF,202001,3.90,2020,1,duty free-711-BAT-B&H FF
1,duty free,711,BAT,B&H FF,202002,2.26,2020,2,duty free-711-BAT-B&H FF
2,duty free,711,BAT,B&H FF,202003,1.98,2020,3,duty free-711-BAT-B&H FF
3,duty free,711,BAT,B&H FF,202004,0.78,2020,4,duty free-711-BAT-B&H FF
4,duty free,711,BAT,B&H FF,202005,0.68,2020,5,duty free-711-BAT-B&H FF


In [26]:
# Only taking data where atleast 6 months of data is giving
df_ls['cnt'] = df_ls.groupby(['uid'])['YYYYMM'].transform('size')
display(df_ls['cnt'].value_counts())

# Atleast we need 6 months data 
df_ls = df_ls[df_ls['cnt'] >=6]
df_ls.drop('cnt', axis=1, inplace=True)
print('Unique UIDs - ', df_ls['uid'].nunique())

12    39264
6       960
11      627
8       624
7       441
10      390
9       171
Name: cnt, dtype: int64

Unique UIDs -  3688


In [27]:
# #Set the forecast horizon wich is equal to test horizon
h=3
#Running through all the Key Account and PRoductID
final_prediction = pd.DataFrame()

for i in df_ls['uid'].unique():
    print('UID: ', i)
    da = df_ls[df_ls.uid == i].copy()
    da = da.reset_index(drop=True)
    print("Total data shape: ", da.shape)
    

    #Creating Train and test
    data_ts1=da[['actual_vol']].reset_index(drop = True)
    data_ts = data_ts1.actual_vol
    train= data_ts[:-h]
    print("Train Shape: ", train.shape)
    test = data_ts[-h:]
    print("Test Shape: ", test.shape)

    #Train the Model and test
    data_pred = exp_smoothing_trend(train,extra_periods=h)

    data_pred['demand'] = data_ts1['actual_vol']


    #Calculate performance metrics
    df_per = data_pred.tail(h)
    MSE= mean_squared_error(y_true=df_per['demand'], y_pred=df_per['forecast'])
    MAE= mean_absolute_error(y_true=df_per['demand'], y_pred=df_per['forecast'])
    MAPE= mean_absolute_percentage_error(y_true=df_per['demand'], y_pred=df_per['forecast'])

#         # Creating the model using full data and forecasting n steps  
    data_forcast = exp_smoothing_trend(data_ts,extra_periods=h,alpha=0.4,beta=0.4,phi=0.8)


    data_forcast['uid'] = i
    #Appending Performance metrics to the final frame
    data_forcast['MSE'] = MSE
    data_forcast['MAE'] = MAE
    data_forcast['MAPE'] = MAPE
    data_forcast['Acc'] = 100 - MAPE
    data_forcast = data_forcast.reset_index(drop=True)   

    # Adding te time
    data_forcast['Year'] = da['year'].copy()
    data_forcast['Month'] = da['month'].copy()

    # Null value index
    null_values = data_forcast[data_forcast['Month'].isnull()].index
    print("Null Values ", null_values)
    count=0

    for loc in null_values:
        if data_forcast.loc[loc-1,'Month'] < 12:
            data_forcast.loc[loc,"Month"] = data_forcast.loc[loc-1,'Month']+1
            data_forcast.loc[loc,"Year"] = data_forcast.loc[loc-1,"Year"]
        else:
            data_forcast.loc[loc,"Month"] = 1+count
            count=count+1
            if count == 12:
                count=0
            data_forcast.loc[loc,"Year"] =2021

    data_forcast['State'] = data_forcast['uid'].apply(lambda x: x.split("-")[0])
    data_forcast['ACC'] = data_forcast['uid'].apply(lambda x: x.split("-")[1])
    data_forcast['Company'] = data_forcast['uid'].apply(lambda x: x.split("-")[2])
    data_forcast['SKU'] = data_forcast['uid'].apply(lambda x: x.split("-")[3])

    final_prediction = pd.concat([final_prediction, data_forcast],
                                    axis=0,
                                    ignore_index=True
                                   )
    print("-*"*50)


UID:  duty free-711-BAT-B&H FF
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  duty free-711-BAT-DH FF DKS
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  duty free-711-BAT-DH FF DKS Empire
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  duty free-711-BAT-DH LTS DKS
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  duty free-711-PMI-MARL MTH Double Burst
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  duty free-711-PMI-MARL MTH Ice
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  duty free-711-PMI-SAMPOERNA FF A Mild
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  duty free-711-PMI-SAMPOERNA MTH A Mild
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Valu

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  ipoh-711-JTI-LD MTH Purple
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  ipoh-711-JTI-LD MTH Zoom
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  ipoh-711-JTI-M7 LTS Charcoal
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  ipoh-711-JTI-M7 LTS Sky Blue
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='in

Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  ipoh-BHP-BAT-KENT FF
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  ipoh-BHP-BAT-KENT LTS Futura
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  ipoh-BHP-BAT-KYO FF
Total data shape:  (6, 9)
Train Shape:  (3,)
Test Shape:  (3,)
Null Values  Int64Index([6, 7, 8], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  ipoh-BHP-BAT-KYO LTS
Total data shape:  (6, 9)
Train Shape:  (3,)
Test Shape:  (3,)
Null Values  Int64

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  ipoh-BHP-PMI-CHESTERFIELD MTH Purple
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  ipoh-BHP-PMI-L&M FF
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  ipoh-BHP-PMI-L&M LTS Charcoal
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  ipoh-BHP-PMI-L&M MTH
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int6

...

Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  melaka-Others-BAT-PM LTS
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  melaka-Others-BAT-PM MTH
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  melaka-Others-BAT-PM MTH Ice
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  melaka-Others-BAT-PM MTH 

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  melaka-Others-PMI-MARL MTH
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  melaka-Others-PMI-MARL MTH Black Mth
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  melaka-Others-PMI-MARL MTH Double Burst
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  melaka-Others-PMI-MARL MTH Ice
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Inde

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  penang-Others-JTI-M7 LTS Charcoal
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  penang-Others-JTI-M7 LTS Sky Blue
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  penang-Others-JTI-M7 LTS Wind Blue
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  penang-Others-JTI-M7 MTH
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  selangor-Others-BAT-KENT LTS Neo
Total data shape:  (9, 9)
Train Shape:  (6,)
Test Shape:  (3,)
Null Values  Int64Index([9, 10, 11], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  selangor-Others-BAT-PM FF
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  selangor-Others-BAT-PM LTS
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  selangor-Others-BAT-PM MTH
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='i

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  selangor-Others-PMI-MARL FF
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  selangor-Others-PMI-MARL LTS
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  selangor-Others-PMI-MARL MTH
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13, 14], dtype='int64')
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
UID:  selangor-Others-PMI-MARL MTH Black Mth
Total data shape:  (12, 9)
Train Shape:  (9,)
Test Shape:  (3,)
Null Values  Int64Index([12, 13,

In [28]:
final_prediction.to_csv('../Output/final_pred.csv', index=False)

# Forecasting using Arima

In [7]:
# Taking data which has full data
final_ka04_df['cnt'] = final_ka04_df.groupby(['uid'])['YYYYMM'].transform('size')
display(final_ka04_df['cnt'].value_counts())

# Atleast we need 6 months data 
full_data = final_ka04_df[final_ka04_df['cnt'] ==12].copy()
full_data.drop('cnt', axis=1, inplace=True)
print('After selectign Unique UIDs - ', full_data['uid'].nunique())
full_data.shape

12    39264
6       960
3       705
11      627
8       624
5       445
7       441
10      390
2       224
4       224
9       171
1       144
Name: cnt, dtype: int64

After selectign Unique UIDs -  3272


(39264, 9)

In [58]:
# Import the library
from pmdarima import auto_arima
rmse_list = []
uid_list = []
h=3 # No of periods to forecast

# Fit auto_arima function to AirPassengers dataset
for i in ['selangor-711-BAT-DH FF DKS Empire']: #full_data['uid'].unique():
    
    ts = full_data.loc[full_data['uid'] == i, 
                       ['actual_vol', 'YYYYMM']]
    
    ts['YYYYMM'] = pd.to_datetime(ts['YYYYMM'], format="%Y%m")
    ts.index = ts['YYYYMM']
    ts.index = ts.index.to_period('M')
    ts.drop('YYYYMM', axis=1, inplace=True)
#     print(ts.head())
    
    # Split the data into train and test    
    train = ts[:-h].values
    test = ts[-h:].values
    
    print('Train shape: ', train.shape)
    print('Test Shape: ', test.shape)
    
    history = [x for x in train]
    predictions = []
    
    for t in range(len(test)):
        model = ARIMA(history, order=(3,1,1))
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
#         print('predicted=%f, expected=%f' % (yhat, obs))
        
    # evaluate forecasts
    rmse = np.sqrt(mean_squared_error(test, predictions))
    print('Test RMSE: %.3f' % rmse)
    
    uid_list.append(i)
    rmse_list.append(rmse)
    print("-*"*50)
    
err_res = pd.DataFrame({'uid': uid_list, 'rmse':rmse_list})
    
    

Train shape:  (9, 1)
Test Shape:  (3, 1)


e:\support project\retail_items_forecasting\venv\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Test RMSE: 222.068
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*


In [49]:
err_res.to_csv('../Output/ARIMA_1_2_1.csv', index=False)

PermissionError: [Errno 13] Permission denied: '../Output/ARIMA_1_2_1.csv'

In [3]:
final_ka04_df.head()

,DWH_STATE_Cleaned,ACC,Company,SKU_CODE,YYYYMM,actual_vol
0,duty free,711,BAT,B&H FF,202001,3.90
1,duty free,711,BAT,B&H FF,202002,2.26
2,duty free,711,BAT,B&H FF,202003,1.98
3,duty free,711,BAT,B&H FF,202004,0.78
4,duty free,711,BAT,B&H FF,202005,0.68
